# 2.2 Chat Completion Agent with Semantic Kernel

A Chat Completion Agent in Semantic Kernel provides a streamlined way to handle function calling with automatic execution. This is equivalent to LangChain's ReAct Agent but uses Semantic Kernel's ChatCompletionAgent, which automatically handles the conversation flow and function execution behind the scenes.

In [ ]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai.services.azure_chat_completion import AzureChatCompletion
from semantic_kernel.contents import ChatHistory
from semantic_kernel.functions import kernel_function
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from typing import Annotated

Create the kernel and configure Azure OpenAI service

In [ ]:
# Create a kernel instance
kernel = Kernel()

# Add Azure OpenAI chat completion service
service_id = "azure_openai"
kernel.add_service(
    AzureChatCompletion(
        service_id=service_id,
        deployment_name="gpt-4.1-mini",
    )
)

Define the weather tool as a Semantic Kernel function

In [ ]:
class WeatherPlugin:
    """
    A plugin that provides weather information for specific cities.
    """
    
    @kernel_function(
        description="Get the current weather for a specified location",
        name="get_weather"
    )
    def get_weather(
        self,
        location: Annotated[str, "The name of the city to get the weather for. Must be one of 'Chicago', 'New York', or 'Los Angeles'"]
    ) -> Annotated[str, "A string describing the current weather in the specified location"]:
        """
        Get the current weather for a specified location.
        """
        weather_data = {
            "New York": "Sunny, 25°C",
            "Los Angeles": "Cloudy, 22°C",
            "Chicago": "Rainy, 18°C"
        }
        return weather_data.get(location, "Weather data not available for this location.")

# Add the weather plugin to the kernel
kernel.add_plugin(WeatherPlugin(), plugin_name="weather")

Create the Chat Completion Agent

In [ ]:
# Create a ChatCompletionAgent with function calling enabled
agent = ChatCompletionAgent(
    service_id=service_id,
    kernel=kernel,
    name="WeatherAgent",
    instructions="You are a helpful assistant that can provide weather information for specific cities.",
    execution_settings={
        service_id: kernel.get_prompt_execution_settings_from_service_id(service_id)
    }
)

# Enable automatic function calling
agent.execution_settings[service_id].function_choice_behavior = FunctionChoiceBehavior.Auto()

Semantic Kernel agents provide a more structured approach to conversation management. Let's visualize the agent's capabilities:

In [ ]:
# Display agent information
print(f"Agent Name: {agent.name}")
print(f"Agent Instructions: {agent.instructions}")
print(f"Available Functions:")
for plugin_name in kernel.plugins:
    plugin = kernel.plugins[plugin_name]
    for function_name in plugin:
        function = plugin[function_name]
        print(f"  - {plugin_name}.{function_name}: {function.description}")

Create chat history and interact with the agent

In [ ]:
# Create chat history
chat_history = ChatHistory()

# Add user message
chat_history.add_user_message("What is the weather like in NYC?")

Have the agent respond to the user query

In [ ]:
# Get agent response
async for response in agent.invoke(chat_history):
    chat_history.add_message(response)
    break  # Get the first (and likely only) response

Print the conversation

In [ ]:
# Print the conversation
for message in chat_history.messages:
    role = message.role.value.title()
    content = str(message.content)
    print(f"================================ {role} Message ================================")
    print()
    print(content)
    print()

## Multi-turn Conversation Example

Let's demonstrate how the agent maintains context across multiple turns:

In [ ]:
# Continue the conversation
chat_history.add_user_message("What about Chicago?")

# Get agent response
async for response in agent.invoke(chat_history):
    chat_history.add_message(response)
    break

# Add another question
chat_history.add_user_message("Which city has the warmest weather?")

# Get final response
async for response in agent.invoke(chat_history):
    chat_history.add_message(response)
    break

In [ ]:
# Print the full conversation
print("=== Full Conversation ===")
for i, message in enumerate(chat_history.messages, 1):
    role = message.role.value.title()
    content = str(message.content)
    print(f"[{i}] {role}: {content}")
    print()

## Key Differences from LangChain ReAct Agent

1. **Agent-First Design**: Semantic Kernel's `ChatCompletionAgent` is designed as a first-class citizen that encapsulates instructions, execution settings, and behavior.

2. **Automatic Flow Management**: The agent automatically handles the conversation flow, function calling, and response generation without manual intervention.

3. **Streaming Support**: The `invoke` method returns an async generator, allowing for streaming responses and better control over conversation flow.

4. **Built-in Context Management**: The agent maintains conversation context automatically and can handle multi-turn conversations seamlessly.

5. **Execution Settings Integration**: Agent-specific execution settings are cleanly integrated, making it easy to configure behavior per agent.

6. **Plugin Integration**: The agent automatically discovers and can use any functions from plugins added to the kernel.

7. **Simplified API**: Compared to LangChain's ReAct agent setup, Semantic Kernel provides a more straightforward API for creating and using agents.

This approach provides better encapsulation and makes it easier to build complex multi-agent systems where each agent has specific roles and capabilities.